In [1]:
!nvidia-smi

Mon May  3 02:48:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%load_ext tensorboard

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import transformers
import torchmetrics
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
CONFIG={
    'DATA_PATH':'../../datasets/concept_relationship_prime.csv', # PATH TO UNZIP DATASET
    'concept1':'concept_id_1',
    'concept2':'concept_id_2',
    'labels':'relationship_id',
    'SEED':13,
    'MAX_LEN':256,
    'MODEL_NAME_OR_PATH':'dmis-lab/biobert-v1.1',
    'LEARNING_RATE':2e-5,
    'ADAM_EPSILON':1e-8,
    'WEIGHT_DECAY':0.0,
    'NUM_CLASSES':30,
    'TRAIN_BS':32,
    'VAL_BS':32,
    'WARMUP_STEPS':0,
    'MAX_EPOCHS':5,
    'CHECKPOINT_DIR':'./checkpoints',
    'NUM_WORKERS':8,
    'PRECISION':16,
    'MODEL_SAVE_NAME':'biobert_v1'
}

In [5]:
_=pl.seed_everything(CONFIG['SEED'])

Global seed set to 13


In [6]:
class ConceptLearningDataset(torch.utils.data.Dataset):

    def __init__(self, max_len: int, tokenizer, concept1, concept2, labels):
        super().__init__()
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.concept1 = concept1
        self.concept2 = concept2
        self.labels = labels

    def __len__(self):
        return len(self.concept1)

    def __getitem__(self, idx):
        concept_1 = self.concept1[idx]
        concept_2 = self.concept2[idx]
        encoded_input = self.tokenizer.encode_plus(
            text=concept_1,
            text_pair=concept_2,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'labels': torch.tensor(self.labels[idx]),
            'input_ids': encoded_input['input_ids'].view(-1),
            'attention_mask': encoded_input['attention_mask'].view(-1),
            'token_type_ids': encoded_input['token_type_ids'].view(-1),
        }

In [7]:
class ConceptLearningDataModule(pl.LightningDataModule):

    def __init__(self):
        super().__init__()

    def prepare_data(self):
      self.data_df=pd.read_csv(CONFIG['DATA_PATH'],nrows=40_000)
      print("Types of Relationship ",self.data_df['relationship_id'].unique().shape)
      self.label_encoder=LabelEncoder()
      self.data_df[CONFIG['labels']]=self.label_encoder.fit_transform(self.data_df[CONFIG['labels']])
      self.tokenizer=transformers.AutoTokenizer.from_pretrained(CONFIG['MODEL_NAME_OR_PATH'])

    def setup(self, stage):
      if stage=='fit':
        self.train_df,self.val_df=train_test_split(self.data_df,random_state=CONFIG['SEED'],test_size=0.1)

    def get_dataset(self,df):
      dataset = ConceptLearningDataset(max_len=CONFIG['MAX_LEN'],
                               tokenizer=self.tokenizer,
                               concept1=df[CONFIG['concept1']].values,
                               concept2=df[CONFIG['concept2']].values,
                               labels=df[CONFIG['labels']].values)
      return dataset

    def train_dataloader(self):
      train_dataset=self.get_dataset(self.train_df)
      train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                                     batch_size=CONFIG['TRAIN_BS'], 
                                                     shuffle=True, 
                                                     num_workers=CONFIG['NUM_WORKERS'])
      
      return train_dataloader

    def val_dataloader(self):
      val_dataset=self.get_dataset(self.val_df)
      val_dataloader = torch.utils.data.DataLoader(val_dataset, 
                                                     batch_size=CONFIG['VAL_BS'], 
                                                     shuffle=False, 
                                                     num_workers=CONFIG['NUM_WORKERS'])
      
      return val_dataloader

In [8]:
class ConceptLearningModel(pl.LightningModule):

  def __init__(self,model_name_or_path:str,
               num_labels:int,
               learning_rate:float,
               adam_epsilon:float,
               weight_decay:float,
               max_len:int,
               warmup_steps:int,
               gpus:int,max_epochs:int,accumulate_grad_batches:int):
    super().__init__()
    self.model_name_or_path=model_name_or_path
    self.num_labels=num_labels
    
    self.save_hyperparameters('learning_rate','adam_epsilon','weight_decay','max_len','gpus','accumulate_grad_batches','max_epochs','warmup_steps') 

    self.config = transformers.AutoConfig.from_pretrained(model_name_or_path, num_labels=self.num_labels)
    self.model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name_or_path, config=self.config)
    metrics = torchmetrics.MetricCollection([
        torchmetrics.Accuracy(),
        torchmetrics.F1(num_classes=CONFIG['NUM_CLASSES'])
      ]
    )
    self.train_metrics=metrics.clone()
    self.val_metrics=metrics.clone()


  def forward(self,inputs):
    return self.model(**inputs)
  
  def training_step(self,batch,batch_idx):
    loss,logits=self(batch)[:2]
    predictions=torch.argmax(logits,dim=1)
    self.train_metrics(predictions,batch['labels'])
    self.log_dict({'train_accuracy':self.train_metrics['Accuracy'],'train_f1':self.train_metrics['F1']}, on_step=False, on_epoch=True)
    return {
        'loss':loss,
        'predictions':predictions,
        'labels':batch['labels']
    }
  
  def validation_step(self,batch,batch_idx):
    loss,logits=self(batch)[:2]
    predictions=torch.argmax(logits,dim=1)
    self.val_metrics(predictions,batch['labels'])
    self.log_dict({'val_accuracy':self.val_metrics['Accuracy'],'val_f1':self.val_metrics['F1']}, on_step=False, on_epoch=True)
    return {
        'loss':loss,
        'predictions':predictions,
        'labels':batch['labels']
    }

  def validation_epoch_end(self,outputs):
    loss=torch.tensor([x['loss'] for x in outputs])
    loss = loss.mean()
    self.log('val_loss', loss, prog_bar=True,on_step=False, on_epoch=True )
  
  def training_epoch_end(self,outputs):
    loss=torch.tensor([x['loss'] for x in outputs])
    loss = loss.mean()
    self.log('train_loss', loss, prog_bar=True,on_step=False, on_epoch=True )
  
  def setup(self, stage):
    if stage == 'fit':
      train_loader = self.train_dataloader()
      self.total_steps = (
          (len(train_loader.dataset) // (train_loader.batch_size * max(1, self.hparams.gpus)))
          // self.hparams.accumulate_grad_batches * float(self.hparams.max_epochs)
      )

  def configure_optimizers(self):
    model = self.model
    no_decay = ["bias", "LayerNorm.weight","LayerNorm.bias"]
    optimizer_grouped_parameters = [
          {
              "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
              "weight_decay": self.hparams.weight_decay,
          },
          {
              "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
              "weight_decay": 0.0,
          },
    ]
    optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    scheduler = transformers.get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=self.total_steps
    )
    scheduler = {
        'scheduler': scheduler,
        'interval': 'step',
        'frequency': 1
    }
    return [optimizer] ,[scheduler]

In [9]:
model_save_checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath=CONFIG['CHECKPOINT_DIR'],
    filename=f"{CONFIG['MODEL_SAVE_NAME']}"+'-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
)

In [10]:
trainer = pl.Trainer(gpus=torch.cuda.device_count(),
                     max_epochs=CONFIG['MAX_EPOCHS'],
                     callbacks=[model_save_checkpoint],
                    #  precision=CONFIG['PRECISION'],
                     num_sanity_val_steps=0
                    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [11]:
model=ConceptLearningModel(
    model_name_or_path=CONFIG['MODEL_NAME_OR_PATH'],
    num_labels=CONFIG['NUM_CLASSES'],
    learning_rate=CONFIG['LEARNING_RATE'],
    adam_epsilon=CONFIG['ADAM_EPSILON'],
    weight_decay=CONFIG['WEIGHT_DECAY'],
    max_len=CONFIG['MAX_LEN'],
    warmup_steps=CONFIG['WARMUP_STEPS'],
    max_epochs=trainer.max_epochs,
    gpus=trainer.gpus,
    accumulate_grad_batches=trainer.accumulate_grad_batches,
)

mnli_dm=ConceptLearningDataModule()
trainer.fit(model,mnli_dm)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Types of Relationship  (30,)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                          | Params
----------------------------------------------------------------
0 | model         | BertForSequenceClassification | 108 M 
1 | train_metrics | MetricCollection              | 0     
2 | val_metrics   | MetricCollection              | 0     
----------------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.333   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/va2134/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Validating: 0it [00:00, ?it/s]

/home/va2134/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


AttributeError: 'ConceptLearningDataModule' object has no attribute 'has_teardown_None'

In [12]:
trainer.logged_metrics

{'train_accuracy': tensor(0.7863, device='cuda:0'),
 'train_f1': tensor(0.7863, device='cuda:0'),
 'epoch': tensor(1.),
 'train_loss': tensor(0.5545, device='cuda:0'),
 'val_accuracy': tensor(0.7860),
 'val_f1': tensor(0.7860),
 'val_loss': tensor(0.5301)}

In [13]:
%tensorboard --logdir ./lightning_logs/